# Re-analysis of unassigned reads

It is important to know the identity of unassigned reads for contamination or reference database ambiguities. This code will BLAST unassigned reads against entire GenBank.

Based on the results of the 98% BLAST identity metaBEAT run, a new BIOM table containing only OTUs that were not taxonomically assigned is generated. A fasta file with the corresponding sequences is also prepared.


Required files:

 - fasta file containing all query sequences (global centroids), as produced by 98% identity metaBEAT run

 - taxonomy annotated OTU biom table in json format from a metaBEAT run - not the taxonomy collapsed BIOM table.


Load the necessary functions. Functions are in place as of version '0.97.4-global' (commit: 9110e5a3f4a979e85733f83cb0388b00586544f6).

In [1]:
!pwd

/home/working/Harper_et_al_2018/Jupyter_notebooks


In [2]:
cd ..

/home/working/Harper_et_al_2018


In [3]:
!mkdir 4-unassigned

In [4]:
cd 4-unassigned/

/home/working/Harper_et_al_2018/4-unassigned


In [5]:
import metaBEAT_global_misc_functions as mb

Read in BIOM file from metaBEAT analysis.

In [6]:
table = mb.load_BIOM('../3-taxonomic_assignment/GLOBAL/BLAST_0.98/12S-trim30_crop110_min90_merge-forwonly_nonchimera_c0.97cov5_blast98-id1-OTU-taxonomy.blast.biom', 
                     informat='json')


Specified BIOM input format 'json' - ok!


In [7]:
# Double check that we've got a table
#print table

Extract only OTUs that were not assigned to any taxa i.e. 'unassigned'.

In [8]:
unassigned_table = mb.BIOM_return_by_tax_level(taxlevel='unassigned', BIOM=table, invert=False)

Found taxonomy metadata with OTUs - ok!


In [9]:
# Check metadata in new table to see if we only got the unassigned bits
#print unassigned_table.metadata(axis='observation')

Extract only sequences mentioned in the table.

In [10]:
mb.extract_fasta_by_BIOM_OTU_ids(in_fasta='../3-taxonomic_assignment/GLOBAL/global_queries.fasta', 
                                 BIOM=unassigned_table, 
                                 out_fasta='unassigned_only.fasta')

Looking to extract 7609 sequences
Parsing ../3-taxonomic_assignment/GLOBAL/global_queries.fasta
identified 7609 target sequences .. OK!
Writing sequences to file: unassigned_only.fasta


In [11]:
unassigned_table_notax = mb.drop_BIOM_taxonomy(unassigned_table)

In [12]:
# Double check that the taxonomy is gone
#print unassigned_table_notax.metadata(axis='observation')

Write reduced table without taxonomy metadata, i.e. denovo table, to file

In [13]:
mb.write_BIOM(BIOM=unassigned_table_notax, target_prefix='unassigned_only_denovo', outfmt=['json','tsv'])

Writing 'unassigned_only_denovo.biom'
Writing 'unassigned_only_denovo.tsv'


A `gb_to_taxid.csv` file is required for the unassigned BLAST to match GenBank accession numbers to the corresponding taxid for a given organism. This has been provided for you in the directory containing this Jupyter notebook (`./Jupyter_notebooks/`) but you will have to move it to `../4-unassigned/`.

In [14]:
!cp ../Jupyter_notebooks/gb_to_taxid.csv ../4-unassigned/

Make sure `gb_to_taxid.csv` file is present.

In [16]:
!head gb_to_taxid.csv

KF451804.1,9606
KX211931.1,9597
998452514,9606
KC252392.1,9606
KT749813.1,9606
LC062069.1,10090
930576047,9606
647737363,9615
998452864,9606
KX211933.1,9597


You will need to make sure you have a copy of the NCBI nucleotide database on your computer. Use the command line to download this [here](ftp://ftp.ncbi.nih.gov/).

The files `unassigned_only_denovo.biom` and `unassigned_only.fasta` can be used as input for new metaBEAT run.

In [17]:
!metaBEAT_global.py -h

usage: metaBEAT.py [-h] [-Q <FILE>] [-B <FILE>] [--g_queries <FILE>] [-v] [-s]
                   [-f] [-p] [-k] [-t] [-b] [-m <string>] [-n <INT>] [-E] [-e]
                   [--read_stats_off] [--PCR_primer <FILE>] [--bc_dist <INT>]
                   [--trim_adapter <FILE>] [--trim_qual <INT>] [--phred <INT>]
                   [--trim_window <INT>] [--read_crop <INT>]
                   [--trim_minlength <INT>] [--merge] [--product_length <INT>]
                   [--merged_only] [--forward_only] [--length_filter <INT>]
                   [--length_deviation <FLOAT>] [-R <FILE>] [--gb_out <FILE>]
                   [--rec_check] [--gb_to_taxid <FILE>] [--cluster]
                   [--clust_match <FLOAT>] [--clust_cov <INT>]
                   [--blast_db <PATH>] [--blast_xml <PATH>]
                   [--update_taxonomy] [--taxonomy_db <FILE>]
                   [--min_ident <FLOAT>] [--min_ali_length <FLOAT>]
                   [--bitscore_skim_LCA <FLOAT>] [--bitscore_skim_adju

**GO!**

In [17]:
%%bash

metaBEAT_global.py \
-B unassigned_only_denovo.biom \
--g_queries unassigned_only.fasta \
--blast --blast_db ../../entireGB_DB/nt/nt --min_ident 0.98 --min_ali_length 0.8 \
-m 12S -n 5 \
-v \
-@ L.Harper@2015.hull.ac.uk \
-o unassigned_only &> log_unassigned

In [18]:
!tail -n 50 log_unassigned

If the analysis breaks, you have to find the OTU and GenBank ('gb') accession number that caused this.

Open the log file and look for the query that was being processed when the analysis broke. The log file will tell you how many gb accession numbers have been seen before until eventually one will cause the pipeline to break. If it has seen the gb accession number before in the first 10 hits for example, it is the gb accession number of the 11th hit that broke the pipeline. Open the `global_blastn.out.xml` in the command line using vim and find the last query that was being processed. Look for the hit with the gb accession number that caused the analysis to break. Copy the gb number and search NCBI with it. The result returned is the species that we need to obtain a taxid for. Click on the species and copy the taxid on the species page. Manually append this taxid, along with the gb number, to your `gb_to_taxid.csv` file in text editor.

Continue to re-run BLAST and repeat trouble-shooting process until metaBEAT completes successfully.